# Lego Ebay Project

In [1]:
import pandas as pd
import numpy as np
import sqlite3 as sql
import matplotlib.pyplot as plt
import datetime
%matplotlib inline

# **********   INJEST   **********
Create a connection to the database and save SQL queries as variables

In [2]:
# Create connection to database
database = "C:\\Users\\zubaz\\Documents\\Python\\EbayLegoWebscrape\\lego.db"
connection = sql.connect(database)

In [3]:
# 2 queries for 2 tables, one with price data, one with set metadata
query1 = '''SELECT item_num, set_num, date, price
            FROM ebay_prices'''
query2 = '''SELECT * FROM set_details'''

### Create dataframe for query1
This dataframe contains the ebay prices data

In [4]:
df = pd.read_sql_query(query1, connection)
# df.head()

In [5]:
df.head()

,item_num,set_num,date,price
0,123456,10256,02-01-2022,20.0
1,111440626278,70147,01-07-2022,158.99
2,111567758032,70222,01-07-2022,119.99
3,111567758519,70223,01-07-2022,184.98
4,111567807708,41052,01-07-2022,185.98


In [6]:
# df.info()

In [7]:
# find how many unique set_num there are
num_of_sets = len(pd.unique(df['set_num']))
print(num_of_sets)


1936


### Create a dataframe for query2
This dataframe contains the set dimension data

In [8]:
df_set = pd.read_sql_query(query2, connection)
# df_set.head()

In [9]:
# close connection to database
connection.close()

Get some basic date and set data

In [10]:
#change date column from text to datetime
df['date'] = pd.to_datetime(df['date'])

In [11]:
# find earliest and latest dates and how many days of data exists
oldest_date = df['date'].min()
recent_date = df['date'].max()
date_difference = recent_date - oldest_date
num_of_rows = len(df.index)
print(f'Earliest price data: {oldest_date}')
print(f'Latest price data: {recent_date}')
print(f'Days of data: {date_difference}')
print(f"Price data for {num_of_sets} Lego sets")
print(f'Total rows: {num_of_rows}')

Earliest price data: 2022-01-07 00:00:00
Latest price data: 2022-03-21 00:00:00
Days of data: 73 days 00:00:00
Price data for 1936 Lego sets
Total rows: 101215


# **********  EXPLORE & CLEAN  **********

## Price Dataframe
Examine the 'price' column and do some cleaning

In [12]:
# Check if all values in price column are integers
all(x.is_integer() for x in df['price'])

False

In [13]:
print(df[pd.to_numeric(df['price'], errors='coerce').isnull()])

            item_num set_num       date            price
30      112575971218   75523 2022-01-19  25.79 to 135.81
42      112919829075   41231 2022-02-23    4.98 to 16.59
43      112939172066   10235 2022-02-06     3.29 to 4.98
44      113052454430   60153 2022-03-07     4.39 to 5.98
49      113301537932   10695 2022-01-21  16.37 to 122.89
...              ...     ...        ...              ...
100031  403415685071   10251 2022-01-22         1,286.32
100105  403424279876   21137 2022-01-22         1,749.99
100297  403443532968   10251 2022-02-06         1,081.17
100714  403496025853   70618 2022-02-24         2,000.00
101102  403538479899   10294 2022-03-15         1,000.00

[769 rows x 4 columns]


In [14]:
# looks like the commas are bad, let's replace commas with nothing
df['price'] = df['price'].replace(',','', regex=True)

In [15]:
# check to see what the rest of the non numeric values look like
# print(df['price'] [pd.to_numeric(df['price'], errors='coerce').isnull()])

In [16]:
df.shape

(101215, 4)

In [17]:
# find the index of rows with 'to' in them
# these values are too hard to deal with, probably not representative listings
remove_rows = df[df['price'].str.contains("to") == True].index

print(remove_rows)

Int64Index([   30,    42,    43,    44,    49,   115,   131,  4281,  4297,
             4303,
            ...
            79352, 79367, 82920, 90841, 90843, 90844, 90845, 91176, 93667,
            93671],
           dtype='int64', length=105)


In [18]:
# remove these rows
df.drop(remove_rows, inplace=True)

In [19]:
df.shape

(101110, 4)

In [20]:
print(df[pd.to_numeric(df['price'], errors='coerce').isnull()])

           item_num set_num       date       price
4707   124363674943   75828 2022-03-14   AU 189.99
11247  134013007163   10229 2022-03-07  GBP 353.27
12135  134041978929   75042 2022-03-04   GBP 40.00
12136  134042010356   75042 2022-03-04   GBP 35.00
12144  134042241968   21146 2022-03-04  EUR 320.22
12153  134042440562   75042 2022-03-04   GBP 25.00
12833  143686166949   10760 2022-03-14   EUR 34.99
16151  153758043579   10760 2022-03-14   EUR 34.99
29359  184057069459   10760 2022-03-14   EUR 27.95
30527  185182410200   10760 2022-03-14   EUR 49.99
33483  193414589158   42108 2022-03-14   AU 250.00
33784  193904131315   10760 2022-03-14    AU 94.99
43476  224806801000   42108 2022-03-14    EUR 8.90
45323  224884348131   75828 2022-03-14  EUR 119.00
45611  232017437466   75828 2022-03-14   GBP 24.99
49597  254382734541   10760 2022-03-14   EUR 44.44
49627  254486995919   75828 2022-03-14  EUR 166.66
49830  254753688791   42108 2022-03-14  EUR 155.55
49886  254789090411   75828 202

In [21]:
# there is a problem with my scraping, it's collecting GBP and EUR values
# I will need to check on this, for now just filter non-digit rows out
df = df[pd.to_numeric(df['price'], errors='coerce').notnull()]
df.head()

,item_num,set_num,date,price
0,123456,10256,2022-02-01,20.0
1,111440626278,70147,2022-01-07,158.99
2,111567758032,70222,2022-01-07,119.99
3,111567758519,70223,2022-01-07,184.98
4,111567807708,41052,2022-01-07,185.98


In [22]:
# checks price column to make sure all rows are numeric
pd.to_numeric(df['price'], errors='coerce').notnull().all()

True

In [23]:
df.dtypes

item_num             int64
set_num             object
date        datetime64[ns]
price               object
dtype: object

In [24]:
# now that all rows in price column are numeric, change column data type to numeric
# pandas will pick int64 if there are no decimals, float 64 if decimals are present, which there are
df['price'] = pd.to_numeric(df['price'])

# change item_num to text so it doesn't use scientific notation later on
# df['item_num'] = df['item_num'].astype('object')
df.dtypes

item_num             int64
set_num             object
date        datetime64[ns]
price              float64
dtype: object

## Metadata Dataframe

In [25]:
# time to clean the set_details data
df_set.head()

,set_num,set_name,theme_group,theme,subtheme,year_released,launch_exit,pieces,minifigs,designer,msrp,age_range,packaging,availability,rating
0,60009-1,Helicopter Arrest,Modern day,City,Police,2013,None,352.0,5,Henrik Andersen,£39.99 / $49.99,5 - 12,Box,Retail - limited,✭✭✭✭✩ 3.8 52 ratings
1,60012-1,Coast Guard 4x4 & Diving Boat,Modern day,City,Coast Guard,2013,01 Jun 2013 - 31 Jul 2015,128.0,2 (2 Unique to this set),None,£9.99 / $19.99 / 12.99€,5 - 12,Box,Retail,✭✭✭✭✩ 3.8 145 ratings 4 Reviews Official Brick...
2,75975-1,Watchpoint: Gibraltar,Licensed,Overwatch,None,2019,01 Jan 2019 - 31 Jul 2020,730.0,4 (3 Unique to this set),Mark Stafford,£79.99 / $89.99 / 87.72€,9+,Box,Retail,✭✭✭✭✩ 3.9 114 ratings 1 Review Official Bricks...
3,10155-1,Maersk Line Container Ship,Model making,Creator Expert,Maersk,2010,01 Aug 2010 - 31 Jul 2011,990.0,None,None,£102.99 / $119.99,8+,Box,LEGO exclusive,✭✭✭✭✩ 4.1 28 ratings 3 Reviews
4,10210-1,Imperial Flagship,Model making,Creator Expert,Miscellaneous,2010,01 Jan 2010 - 31 Dec 2011,1664.0,9 (6 Unique to this set),Raphael Pretesacque,£142.99 / $179.99,14+,Box,Retail - limited,✭✭✭✭✭ 4.6 171 ratings 16 Reviews


In [26]:
# remove -1 from set_num
df_set['set_num'] = df_set['set_num'].str.split('-', n=1).str[0]

In [27]:
# split launch_exit column into 2 columns
df_set[['launch_date', 'retirement_date']] = df_set['launch_exit'].str.split(' - ', expand=True)

In [28]:
df_set.drop('launch_exit', axis=1, inplace=True)

In [29]:
# need to plit the minifigs column into total and unique
df_set[['minifigs_total', 'minifigs_unique']] = df_set['minifigs'].str.split(' ', n=1, expand=True)

In [30]:
# parse the minifig value from minifigs_unique
df_set['minifigs_unique'] = df_set['minifigs_unique'].str.split(' ', n=1).str[0].str.replace('(', '', regex=True)

In [31]:
df_set.drop('minifigs', axis=1, inplace=True)

In [32]:
# parse rating column to get the text after the stars, this grabs non numeric values for rows with no rating
df_set['rating'] = df_set['rating'].str.split(' ', n=2).str[1].str.strip()

In [33]:
# a good way to replace non-numeric rows is to_numeric method, must use coerce to force NaN values for non-numerics
df_set['rating'] = pd.to_numeric(df_set['rating'], errors='coerce')

In [34]:
# change launch retirement date columns to date
# first need to remove spaces
df_set['launch_date'] = df_set['launch_date'].str.replace(' ', '')
df_set['launch_date'] = pd.to_datetime(df_set['launch_date'], format='%d%b%Y', errors='coerce')

In [35]:
# had some t.b.a text for some rows, they threw an error, added errors=coerce and seemed to fix it
df_set['retirement_date'] = df_set['retirement_date'].str.replace(' ', '')
df_set['retirement_date'] = pd.to_datetime(df_set['retirement_date'], format='%d%b%Y', errors='coerce')

In [36]:
# clean up msrp column and grab USD values only - drop pounds and euro values if present
# use regex to extract everything after the $, the dot, and the remaining digits
# REGEX sucks. But remember to use regex101.com, it's a life saver
df_set['msrp'] = df_set['msrp'].str.extract(r"\$(\d+\.\d+)")

In [37]:
# if launch_date is empty add the value from year released date column, 
df_set['launch_date'] = df_set['launch_date'].fillna(df_set['year_released'])
# very cool, it added jan 1 to the year automatically.

In [38]:
# change some datatypes
df_set['year_released'] = df_set['year_released'].astype(int)
df_set['msrp'] = df_set['msrp'].astype(float)
df_set['minifigs_total'] = df_set['minifigs_total'].astype(float)
df_set['minifigs_unique'] = df_set['minifigs_unique'].astype(float)
df_set['set_num'] = df_set['set_num'].astype(int)

In [39]:
df_set.dtypes

set_num                     int32
set_name                   object
theme_group                object
theme                      object
subtheme                   object
year_released               int32
pieces                    float64
designer                   object
msrp                      float64
age_range                  object
packaging                  object
availability               object
rating                    float64
launch_date        datetime64[ns]
retirement_date    datetime64[ns]
minifigs_total            float64
minifigs_unique           float64
dtype: object

In [40]:
# set the index to set_num, which should be unique
df_set.set_index('set_num')

,set_name,theme_group,theme,subtheme,year_released,pieces,designer,msrp,age_range,packaging,availability,rating,launch_date,retirement_date,minifigs_total,minifigs_unique
set_num,,,,,,,,,,,,,,,,
60009,Helicopter Arrest,Modern day,City,Police,2013,352.0,Henrik Andersen,49.99,5 - 12,Box,Retail - limited,3.8,2013-01-01,NaT,5.0,NaN
60012,Coast Guard 4x4 & Diving Boat,Modern day,City,Coast Guard,2013,128.0,None,19.99,5 - 12,Box,Retail,3.8,2013-06-01,2015-07-31,2.0,2.0
75975,Watchpoint: Gibraltar,Licensed,Overwatch,None,2019,730.0,Mark Stafford,89.99,9+,Box,Retail,3.9,2019-01-01,2020-07-31,4.0,3.0
10155,Maersk Line Container Ship,Model making,Creator Expert,Maersk,2010,990.0,None,119.99,8+,Box,LEGO exclusive,4.1,2010-08-01,2011-07-31,NaN,NaN
10210,Imperial Flagship,Model making,Creator Expert,Miscellaneous,2010,1664.0,Raphael Pretesacque,179.99,14+,Box,Retail - limited,4.6,2010-01-01,2011-12-31,9.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9526,Palpatine's Arrest,Licensed,Star Wars,Episode III,2012,645.0,None,89.99,9 - 14,Box,Retail,4.4,2012-08-01,2013-12-31,6.0,6.0
9585,WeDo Resource Set,Educational,Education,None,2011,326.0,None,49.95,7+,Tub,Educational,NaN,2011-01-01,NaT,NaN,NaN
9594,Green City Challenge Set,Educational,Education,Mindstorms,2011,1365.0,None,174.95,None,None,None,NaN,2011-01-01,NaT,1.0,1.0


In [41]:
# check for all unique rows in set_num
# is_unique method only works for a series, create that first and then check for uniqueness
set_num_series = df_set['set_num'].squeeze()
set_num_series.is_unique

True

Loop through a list of sets and calculate and remove outliers, then groupby and average daily price
Create a new data frame that we will use for the rest of the analysis that is now at a daily granularity

In [42]:
# create function to remove outliers using IQR method
def remove_outliers(dataframe):
    """
    Function to identify and remove outliers from the price column using IQR method

    Parameters:
    ----------
    dataframe : a dataframe

    Returns:
    --------
    dataframe with outliers removed

    """
    try:
        Q1 = dataframe['price'].quantile(0.25)
        Q3 = dataframe['price'].quantile(0.75)
        IQR = Q3 - Q1
        lower_lim = Q1 - 1.5*IQR
        upper_lim = Q3 + 1.5*IQR
        outliers_15_low = ( dataframe['price'] < lower_lim)
        outliers_15_high = ( dataframe['price'] > upper_lim)
        df_outliers_removed = dataframe[~(outliers_15_low | outliers_15_high)]
    except:
        pass

    return df_outliers_removed
    
    # I could add another parameter for column name to make this more generic

In [43]:
# create list
# I can't figure out how to import search_sets.py from another folder.
# So I'll just recreate it here

"""Set List Generator

This script reads a csv file that contains set numbers
and creates a list called search_terms that is used in main
to iterate over. Also removes duplicates.
"""


def create_search_list():
    import csv

    search_terms = []
    search_file = 'list_of_all_sets.csv'

    with open(search_file, newline='') as csvfile:
        for row in csv.reader(csvfile):
            search_terms.append(row[0])

    # Remove duplicates
    search_terms = list(dict.fromkeys(search_terms))

    # print(search_terms)
    return search_terms

In [44]:
# I had a major problem with the loop not working, turns out the search_terms 
# is a list of strings and the df we are looping through has set_num as int
# So I cast int to all the value in the for loop below
# But this removes the COMCON and other string set numbers, I should change the set_num
# back to string and remove it from the index in the above cells.
list_of_sets = create_search_list()

# change all list intems to integer
new_list = []
for value in list_of_sets:
    try:
        new_list.append(int(value))
    except:
        pass

# print(new_list)


In [45]:
# Test list of sets to loop through
# test_list = [75827, 75192, 70222, 70223]

# create an empty data frame to append each looped df to
looped_df = pd.DataFrame()
for set in new_list:
    filter = (df['set_num'] == set)
    df_filtered = df[filter]
    df_no_outliers = remove_outliers(df_filtered)
    
    # reset index fills down the set_num column, creates a full dataframe
    df_filtered_grouped = df_no_outliers.groupby(['set_num', 'date']).mean().reset_index()
    looped_df = looped_df.append(df_filtered_grouped)


In [46]:
looped_df.head(50)

,set_num,date,item_num,price
0,76905,2022-02-23,2.645504e+11,73.262800
1,76905,2022-03-04,1.853235e+11,62.990000
2,76905,2022-03-06,2.248710e+11,29.990000
3,76905,2022-03-08,2.696693e+11,45.485000
4,76905,2022-03-11,2.554294e+11,50.000000
5,76905,2022-03-14,2.152332e+11,56.973333
6,76905,2022-03-16,1.653861e+11,86.110000
7,76905,2022-03-18,2.554414e+11,50.000000
8,76905,2022-03-21,2.847046e+11,75.050000
0,76939,2022-02-23,2.458726e+11,52.056364


### Write clean price data to db
Write to database a clean dataset of sales data that is loaded into Power BI for analysis.
Much easier to do the clean up and outlier detection in python.
Also easier to import from SQLite db into Power BI than do the csv dance.

# **********  MERGE  **********

In [ ]:
# check data types before the merge
df_set.dtypes

In [ ]:
looped_df['set_num'] = looped_df['set_num'].astype(int)
looped_df.dtypes

In [ ]:
# I want all values in the price df and mrsp from set data
# this is a left join and we use pandas merge method
joined_df = pd.merge(looped_df,
                    df_set[['set_num', 'msrp']],
                    on = 'set_num',
                    how = 'left' 
)
joined_df.head()

In [ ]:
# create calculated column of $ of appreciation for every row
joined_df['appreciation'] = (joined_df['price'] - joined_df['msrp']).round(2)

# create % appreciation calculated column
joined_df['pct_appreciation'] = (joined_df['appreciation'] / joined_df['msrp'] * 100).round()

joined_df.head()

In [ ]:
# filter all dates to last 30 days from today
today = datetime.datetime.now()
days30ago = today - pd.Timedelta(days=30)

joined_df = joined_df.loc[joined_df.date > days30ago]

In [ ]:
# calculate one % appreciation for each set_num
group_df = joined_df[['set_num', 'pct_appreciation']]
group_df = group_df.groupby('set_num').mean().reset_index()
group_df.head()

In [ ]:
# Now I need to merge the grouped % appreciation back into the set metadata df
final_df = pd.merge(df_set,
                    group_df[['set_num', 'pct_appreciation']],
                    on = 'set_num',
                    how = 'left' 
)
# selected_rows = final_df[~final_df['pct_appreciation'].isnull()]
# selected_rows

In [ ]:
final_df

In [ ]:
# save as csv to use in another notebook for playing around with ML
final_df.to_csv('final_cleaned_df.csv')